#D03 KNN import numpy as np import pandas as pd import warnings import
matplotlib.pyplot as plt import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV from
sklearn.ensemble import VotingClassifier, RandomForestClassifier from
sklearn.neighbors import KNeighborsClassifier from sklearn.metrics
import accuracy_score, classification_report, confusion_matrix from
sklearn.preprocessing import StandardScaler, LabelEncoder

warnings.filterwarnings(‘ignore’)

# Load dataset

df = pd.read_csv(“processed_mpg_data.csv”)

# Encode categorical target variable

label_encoder = LabelEncoder() df\[‘mpg_class’\] =
label_encoder.fit_transform(df\[‘mpg_class’\])

# Define features (X) and target (y)

X = df.drop(columns=\[‘mpg_class’\]) y = df\[‘mpg_class’\]

# Scale features

scaler = StandardScaler() X_scaled = scaler.fit_transform(X)

# Split dataset

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
test_size=0.2, random_state=42, stratify=y)

# Optimize KNN using GridSearchCV

param_grid_knn = {‘n_neighbors’: np.arange(1, 20), ‘metric’:
\[‘euclidean’, ‘manhattan’, ‘minkowski’\]} grid_search_knn =
GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5,
scoring=‘accuracy’) grid_search_knn.fit(X_train, y_train)

# Get the best KNN parameters

best_k = grid_search_knn.best_params\_\[‘n_neighbors’\] best_metric =
grid_search_knn.best_params\_\[‘metric’\] print(f”KNN parameters:
k={best_k}, metric={best_metric}“)

# Train KNN with best parameters

knn = KNeighborsClassifier(n_neighbors=best_k, metric=best_metric)
knn.fit(X_train, y_train)

# Train a baseline Random Forest model for Voting Classifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Create Voting Classifier (KNN + Random Forest)

voting_knn = VotingClassifier(estimators=\[(‘KNN’, knn), (‘RF’, rf)\],
voting=‘hard’) voting_knn.fit(X_train, y_train)

# Evaluate KNN & Voting Classifier

models = { “KNN”: knn, “Voting Classifier (KNN + RF)”: voting_knn }

print(“Comparisons (KNN + Voting):”) for name, model in models.items():
y_pred = model.predict(X_test) accuracy = accuracy_score(y_test, y_pred)
print(f”{name} Test Accuracy: {accuracy \* 100:.2f}%“)

# Generate Classification Reports

print(“Reports:”) for name, model in models.items(): y_pred =
model.predict(X_test) print(f” Classification Report:“)
print(classification_report(y_test, y_pred))

# Confusion Matrix for Voting Classifier

y_pred_voting = voting_knn.predict(X_test) cm = confusion_matrix(y_test,
y_pred_voting)

plt.figure(figsize=(6, 5)) sns.heatmap(cm, annot=True, fmt=‘d’,
cmap=‘Blues’, xticklabels=label_encoder.classes\_,
yticklabels=label_encoder.classes\_) plt.xlabel(“Predicted”)
plt.ylabel(“Actual”) plt.title(“Confusion Matrix - Voting Classifier
(KNN + RF)”) plt.show()